In [1]:
%pip install mediapipe


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
%pip install --upgrade jax==0.4.31 jaxlib==0.4.31


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import cv2
import numpy as np
import os
import csv
import matplotlib.pyplot as plt
import shutil
import mediapipe as mp

In [4]:
# Extraer los frames y guardarlos en una carpeta
def ObtenerFrames(rutaVideo, rutaCarpeta, rutaTrazos):
    if not os.path.exists(rutaCarpeta):
        os.mkdir(rutaCarpeta)
        print("Carpeta creada corrrectamente" )
    else:
        print("Ya existe esa carpeta" )
        
        for filename in os.listdir(rutaCarpeta):
            file_path = os.path.join(rutaCarpeta, filename)
            try:
                if os.path.isfile(file_path) or os.path.islink(file_path):
                    os.unlink(file_path)
                elif os.path.isdir(file_path):
                    shutil.rmtree(file_path)
            except Exception as e:
                print('No se pudo eliminar')
                
    # para la carpeta de los trazos
    if not os.path.exists(rutaTrazos):
        os.mkdir(rutaTrazos)
        print("Carpeta creada corrrectamente" )
    else:
        print("Ya existe esa carpeta" )
        
        for filename in os.listdir(rutaTrazos):
            file_path = os.path.join(rutaTrazos, filename)
            try:
                if os.path.isfile(file_path) or os.path.islink(file_path):
                    os.unlink(file_path)
                elif os.path.isdir(file_path):
                    shutil.rmtree(file_path)
            except Exception as e:
                print('No se pudo eliminar')

    cap = cv2.VideoCapture(rutaVideo)
    if not cap.isOpened():
        print("No se puede abrir el video.")
        return
    
    cont_frames = 0
    
    while True:
        ret, frame = cap.read()
        
        if ret:
            if cont_frames % 10 == 0:  
                
                # preprocesamiento 1. equalizacion de histograma a las imagenes
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) # primero se convierte a grises
                equalizado = cv2.equalizeHist(frame)
                
                
                # guardado de las imágenes
                name = 'frame' + str(cont_frames) + '.jpg'
                print(name)
                
                result = cv2.imwrite(os.path.join(rutaCarpeta, name), equalizado)
                if not result:
                    print(f"Error.")
            cont_frames += 1
        else:
            break
    
    cap.release()
    print("Extracción completada")

In [5]:
# extraer los frames de varios videos
for i in range(1,7):
     video = "./dataset/sospechoso/"+str(i)+".mp4"
     frames = './frames/sospechoso/'+str(i)
     csv_file = './csv/sospechoso/'+str(i)+'.csv'
     trazos = './trazos/sospechoso/'+str(i)

     ObtenerFrames(video,frames, trazos)
     
     # Obtiene la lista de frames del video
     dir = frames
     nombres = []
     with os.scandir(dir) as ficheros:
          for fichero in ficheros:
               nombres.append(dir + '/' + fichero.name)
     
     mp_drawing = mp.solutions.drawing_utils
     mp_drawing_styles = mp.solutions.drawing_styles
     mp_pose = mp.solutions.pose

     # Para imágenes estáticas
     IMAGE_FILES = nombres
     BG_COLOR = (192, 192, 192)  # gris

     # Crear un archivo CSV para almacenar las coordenadas
     with open(csv_file, mode='w', newline='') as file:
          writer = csv.writer(file)
          # Escribir encabezados en el archivo CSV
          header = ['image', 'landmark', 'x', 'y', 'z', 'visibility']
          writer.writerow(header)
          
          with mp_pose.Pose(
               static_image_mode=True,
               model_complexity=2,
               enable_segmentation=True,
               min_detection_confidence=0.5) as pose:
               
               for idx, file in enumerate(IMAGE_FILES):
                    image = cv2.imread(file)
                    image_height, image_width, _ = image.shape
                    # Convertir la imagen de BGR a RGB antes de procesarla.
                    results = pose.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

                    if not results.pose_landmarks:
                         continue

                    # Guardar las coordenadas de los landmarks en el archivo CSV
                    for landmark_id, landmark in enumerate(results.pose_landmarks.landmark):
                         writer.writerow([
                              f'frame{idx}.png',  # Nombre de la imagen
                              mp_pose.PoseLandmark(landmark_id).name,  # Nombre del landmark
                              landmark.x * image_width,  # Coordenada x
                              landmark.y * image_height,  # Coordenada y
                              landmark.z,  # Coordenada z
                              landmark.visibility  # Visibilidad del landmark
                         ])

                    annotated_image = image.copy()
                    condition = np.stack((results.segmentation_mask,) * 3, axis=-1) > 0.1
                    bg_image = np.zeros(image.shape, dtype=np.uint8)
                    bg_image[:] = BG_COLOR
                    annotated_image = np.where(condition, annotated_image, bg_image)

                    # Dibujar los landmarks en la imagen
                    mp_drawing.draw_landmarks(
                         annotated_image,
                         results.pose_landmarks,
                         mp_pose.POSE_CONNECTIONS,
                         landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())

                    # Guardar la imagen anotada
                    cv2.imwrite(trazos + f'frame{idx}.png', annotated_image)
     # print(nombres)

Ya existe esa carpeta
Ya existe esa carpeta
frame0.jpg
frame10.jpg
frame20.jpg
frame30.jpg
frame40.jpg
frame50.jpg
frame60.jpg
frame70.jpg
frame80.jpg
frame90.jpg
frame100.jpg
frame110.jpg
frame120.jpg
frame130.jpg
frame140.jpg
frame150.jpg
frame160.jpg
frame170.jpg
frame180.jpg
frame190.jpg
frame200.jpg
frame210.jpg
frame220.jpg
frame230.jpg
frame240.jpg
frame250.jpg
frame260.jpg
frame270.jpg
frame280.jpg
frame290.jpg
frame300.jpg
frame310.jpg
frame320.jpg
frame330.jpg
frame340.jpg
frame350.jpg
frame360.jpg
frame370.jpg
frame380.jpg
frame390.jpg
frame400.jpg
frame410.jpg
frame420.jpg
frame430.jpg
Extracción completada


c:\Users\chj06\AppData\Local\Programs\Python\Python311\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


Ya existe esa carpeta
Ya existe esa carpeta
frame0.jpg
frame10.jpg
frame20.jpg
frame30.jpg
frame40.jpg
frame50.jpg
frame60.jpg
frame70.jpg
frame80.jpg
frame90.jpg
frame100.jpg
frame110.jpg
frame120.jpg
frame130.jpg
frame140.jpg
frame150.jpg
frame160.jpg
frame170.jpg
frame180.jpg
frame190.jpg
frame200.jpg
frame210.jpg
frame220.jpg
frame230.jpg
frame240.jpg
frame250.jpg
frame260.jpg
frame270.jpg
frame280.jpg
frame290.jpg
frame300.jpg
frame310.jpg
frame320.jpg
frame330.jpg
frame340.jpg
frame350.jpg
frame360.jpg
frame370.jpg
frame380.jpg
frame390.jpg
frame400.jpg
frame410.jpg
frame420.jpg
frame430.jpg
frame440.jpg
frame450.jpg
frame460.jpg
frame470.jpg
frame480.jpg
frame490.jpg
frame500.jpg
frame510.jpg
frame520.jpg
frame530.jpg
frame540.jpg
frame550.jpg
frame560.jpg
frame570.jpg
frame580.jpg
frame590.jpg
frame600.jpg
frame610.jpg
frame620.jpg
frame630.jpg
frame640.jpg
frame650.jpg
frame660.jpg
frame670.jpg
frame680.jpg
frame690.jpg
frame700.jpg
frame710.jpg
frame720.jpg
frame730.jpg
frame

In [6]:
# mp_drawing = mp.solutions.drawing_utils
# mp_drawing_styles = mp.solutions.drawing_styles
# mp_pose = mp.solutions.pose

# # Para imágenes estáticas
# IMAGE_FILES = nombres
# BG_COLOR = (192, 192, 192)  # gris

# # Crear un archivo CSV para almacenar las coordenadas
# with open('pose_landmarks.csv', mode='w', newline='') as file:
#     writer = csv.writer(file)
#     # Escribir encabezados en el archivo CSV
#     header = ['image', 'landmark', 'x', 'y', 'z', 'visibility']
#     writer.writerow(header)
    
#     with mp_pose.Pose(
#         static_image_mode=True,
#         model_complexity=2,
#         enable_segmentation=True,
#         min_detection_confidence=0.5) as pose:
        
#         for idx, file in enumerate(IMAGE_FILES):
#             image = cv2.imread(file)
#             image_height, image_width, _ = image.shape
#             # Convertir la imagen de BGR a RGB antes de procesarla.
#             results = pose.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

#             if not results.pose_landmarks:
#                 continue

#             # Guardar las coordenadas de los landmarks en el archivo CSV
#             for landmark_id, landmark in enumerate(results.pose_landmarks.landmark):
#                 writer.writerow([
#                     f'annotated_image{idx}.png',  # Nombre de la imagen
#                     mp_pose.PoseLandmark(landmark_id).name,  # Nombre del landmark
#                     landmark.x * image_width,  # Coordenada x
#                     landmark.y * image_height,  # Coordenada y
#                     landmark.z,  # Coordenada z
#                     landmark.visibility  # Visibilidad del landmark
#                 ])

#             # Opcional: también puedes anotar la imagen como en el código original
#             annotated_image = image.copy()
#             condition = np.stack((results.segmentation_mask,) * 3, axis=-1) > 0.1
#             bg_image = np.zeros(image.shape, dtype=np.uint8)
#             bg_image[:] = BG_COLOR
#             annotated_image = np.where(condition, annotated_image, bg_image)

#             # Dibujar los landmarks en la imagen
#             mp_drawing.draw_landmarks(
#                 annotated_image,
#                 results.pose_landmarks,
#                 mp_pose.POSE_CONNECTIONS,
#                 landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())

#             # Guardar la imagen anotada
#             cv2.imwrite(f'./trazos3/annotated_image{idx}.png', annotated_image)

In [7]:
# # For webcam input:
# cap = cv2.VideoCapture(video)
# with mp_pose.Pose(
#     min_detection_confidence=0.5,
#     min_tracking_confidence=0.5) as pose:
#   while cap.isOpened():
#     success, image = cap.read()
#     if not success:
#       # If loading a video, use 'break' instead of 'continue'.
#       break

#     # To improve performance, optionally mark the image as not writeable to
#     # pass by reference.
#     image.flags.writeable = False
#     image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#     results = pose.process(image)

#     # Draw the pose annotation on the image.
#     image.flags.writeable = True
#     image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
#     mp_drawing.draw_landmarks(
#         image,
#         results.pose_landmarks,
#         mp_pose.POSE_CONNECTIONS,
#         landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())
#     # Flip the image horizontally for a selfie-view display.
#     cv2.imshow('MediaPipe Pose', cv2.flip(image, 1))
#     if cv2.waitKey(5) & 0xFF == 27:
#       break
# cap.release()